In [1]:
import os
import pickle
import numpy as np
from PIL import Image
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt

In [8]:
vertex_list = []
face_list = []

In [9]:
def get_vertex_face(model_path):
    vertex_list = []
    face_list = []
    with open(model_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.startswith("mtllib") or line.startswith("g") or line.startswith("usemtl"):
                continue
            elif line.startswith("v "):
                #print(line.split())
                x = float(line.split(" ")[1])
                y = float(line.split(" ")[2])
                z = float(line.split(" ")[3])
                vertex_list.append([x, y, z])
                continue
            elif line.startswith("f "):
                v1 = int(line.split(" ")[1].split("/")[0])
                v2 = int(line.split(" ")[2].split("/")[0])
                v3 = int(line.split(" ")[3].split("/")[0])
                face_list.append([v1, v2, v3])
                continue
    return vertex_list, face_list

In [11]:
model_list = []
for file in os.listdir("./rotate"):
    if file.startswith("model_name_"):
        print(os.path.join("./0619render", file))
        with open(os.path.join("./0619render", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

precision_constant = 10000
    
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343"
for modelidx, model in enumerate(model_list):
    # [7, 43, 54, 55, 60, 62, 76, 4854, 4842, 4826, 4821, 4820, 7191, 7186, 7173, 7162]
    # [7911, 7958, 4585, 4600, 6329, 6346, 6388, 6414, 6471, 6495]
    if modelidx not in [7911, 7958, 4585, 4600, 6329, 6346, 6388, 6414, 6471, 6495]:
        continue
    os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            realimg_mask = np.array(Image.open(real_img_path))[:, :, 3].reshape([128, 128])
            syn_img_path = "./0619render/out/rotate/" + str(modelidx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if realimg_mask[i, j] < 255 / 2:
                            print("pass!")
                            continue
                        if (synimg[i, j] == 0).any():
                            continue
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(synimg[i, j])
            
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for vertexidx, vertex in enumerate(vertex_list):
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            candidate_color_list = vertex2color[repr((np.array(vertex) * precision_constant).astype(int))]
            mean = np.mean(candidate_color_list, axis=0).astype(int)
            std = np.std(candidate_color_list, axis=0).astype(int)
            for coloridx, candidate_color in enumerate(candidate_color_list):
                if (candidate_color > mean + 1 * std).all() or (candidate_color < mean - 1 * std).all():
                    candidate_color_list[coloridx] = mean
            color = np.mean(candidate_color_list, axis=0).astype(int)
            color_list[vertexidx] = color
    vertex2neighbor = OrderedDict()
    for face in face_list:
        v1, v2, v3 = face[0] - 1, face[1] - 1, face[2] - 1;
        if str(v1) not in vertex2neighbor:
            vertex2neighbor[str(v1)] = set()
        if str(v2) not in vertex2neighbor:
            vertex2neighbor[str(v2)] = set()
        if str(v3) not in vertex2neighbor:
            vertex2neighbor[str(v3)] = set()
        vertex2neighbor[str(v1)].add(v2)
        vertex2neighbor[str(v1)].add(v3)
        vertex2neighbor[str(v2)].update([v1, v3])
        vertex2neighbor[str(v3)].update([v1, v2])
    cnt_loop = 0
    while True:
        cnt_loop += 1
        all_vertex_not_colored = True
        for vertexidx, vertex in enumerate(vertex_list):
            if str(vertexidx) not in vertex2neighbor:
                continue
            if np.sum(color_list[vertexidx]) < 10:
                color = np.array([0, 0, 0])
                cnt = 0
                for vertex in vertex2neighbor[str(vertexidx)]:
                    if np.sum(color_list[vertex]) >= 10:
                        color += color_list[vertex]
                        cnt += 1
                if cnt != 0:
                    color_list[vertexidx] = np.array(color) / cnt
                all_vertex_not_colored = False
        if all_vertex_not_colored:
            break
        if cnt_loop >= 100:
            break
            

    with open("./refined_ply_syn_mindist_propagation_abnormal_mask_" + str(modelidx) + ".ply", "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex " + str(len(vertex_list)) + "\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("element face " + str(len(face_list)) + "\n")
        f.write("property list uint8 uint32 vertex_indices\n")
        f.write("end_header\n")
        for vertexidx, vertex in enumerate(vertex_list):
            color = color_list[vertexidx]
            f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + " " + str(color[0]) + " " + str(color[1]) + " " + str(color[2]) + "\n")
        for face in face_list:
            f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")

./0619render\model_name_0
./0619render\model_name_1000
./0619render\model_name_2000
./0619render\model_name_3000
./0619render\model_name_5000
./0619render\model_name_6000
./0619render\model_name_7000
./0619render\model_name_8000
./0619render\model_name_9000
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

...

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass!
pass

In [6]:
model_list[55]

'82a783a1ea4e34b8d7755e7baabe8a6f'

In [9]:
vertex2neighbor['3334']

{3326, 3327, 3333, 3865, 3867, 3868}

In [2]:
model_list = []
for file in os.listdir("./0619render"):
    if file.startswith("model_name_"):
        print(os.path.join("./0619render", file))
        with open(os.path.join("./0619render", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

syn_loss = np.zeros(shape=2000)
for idx, model in enumerate(model_list):
    #print(model)
    #break
    if idx >= 2000:
        break
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            syn_img_path = "./0619render/out/rotate/" + str(idx * 1000 + az * 10 + el) + ".0.jpg"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            synimg = np.array(Image.open(syn_img_path))
            syn_loss[idx] += np.mean(np.abs(realimg - synimg))
    syn_loss[idx] /= 54

./0619render\model_name_0
./0619render\model_name_1000
./0619render\model_name_2000
./0619render\model_name_3000
./0619render\model_name_4000
./0619render\model_name_5000
./0619render\model_name_6000
./0619render\model_name_7000
./0619render\model_name_8000
./0619render\model_name_9000


In [3]:
np.argsort(syn_loss)

array([ 651, 1839,  136, ...,  521, 1126,  742], dtype=int64)

In [6]:
syn_loss[1839]

16.601181218653547

In [13]:
model_list[263]

'cce0c3ccaf38bdfc56a612a99ee3c15'